In [ ]:
#import statements
import json
import ipywidgets as widgets
import plotly
import pandas as pd
!pip install anytree
from anytree import PostOrderIter
from anytree.importer import DictImporter


In [ ]:
#read file
with open('flare.json') as f:
    js_data = json.loads(f.read())

In [ ]:
#imports dictonary in a tree form
importer = DictImporter()
root = importer.import_(js_data)


size = []
name = []
parent = []
level = []

def format(node):
  for i in node.children:
    #check if node as attribute value
    if hasattr(i, 'value') == False:
      format(i)
    v = i.value
    #check if node parent as attribute value
    if hasattr(i.parent, 'value'):
      i.parent.value += v
    #if node parent doesn't have a value set to same val as child
    elif hasattr(i.parent, 'value')== False:
      i.parent.value = v

    level.append(len(i.ancestors))
    name.append(i.name)
    parent.append(i.parent.name)
    size.append(i.value)
    
format(root)

#append attributes for root
level.append(0)
name.append(root.name)
parent.append("")
size.append(root.value)

#create df
df = pd.DataFrame()
df['parent'] = parent
df['name'] = name
df['value']= size
df['level'] = level

#slider funtion
def update(sliderVal):
  fig = plotly.graph_objs.Figure()
  fig.add_trace(plotly.graph_objs.Treemap(
      labels = df[df['level']<sliderVal]['name'],
      values = df[df['level']<sliderVal]['value'],
      parents = df[df['level']<sliderVal]['parent']  
  ))
  fig.update_traces(root_color="#f1f1f1")
  fig.update_layout(width = 900, height = 900)
  fig.show()

#create slider widget
widgets.interact(update, sliderVal = (0, 5))